In [2]:
import os
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

os.environ['SPARK_HOME']='/Users/audioworkstation/Documents/WORKSPACE/LEARNING/pyspark-training/spark'
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='lab'
os.environ['PYSPARK_PYTHON']='python'


spark = SparkSession.builder.appName("RDD").getOrCreate()

## Using RDDs

In [21]:
rdd = spark.sparkContext.textFile('./data.txt')
# rdd.collect() is a list of 3 items

result_rdd = rdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: x[1], ascending=False)
result_rdd.take(10)

[('the', 12),
 ('of', 7),
 ('a', 7),
 ('in', 5),
 ('distributed', 5),
 ('Spark', 4),
 ('is', 3),
 ('as', 3),
 ('API', 3),
 ('on', 3)]

## Using DataFrames

In [26]:
df = spark.read.text('./data.txt')
result_df = df.selectExpr("explode(split(value, ' ')) as word") \
    .groupBy('word').count().orderBy(desc('count'))
result_df.show()

+-----------+-----+
|       word|count|
+-----------+-----+
|        the|   12|
|         of|    7|
|          a|    7|
|         in|    5|
|distributed|    5|
|      Spark|    4|
|        API|    3|
|        RDD|    3|
|         is|    3|
|         on|    3|
|    Dataset|    3|
|         as|    3|
|       data|    2|
|   programs|    2|
|        its|    2|
|       API.|    2|
|        and|    2|
|  MapReduce|    2|
|       RDDs|    2|
|        The|    2|
+-----------+-----+
only showing top 20 rows



In [27]:
result_df.take(5)

[Row(word='the', count=12),
 Row(word='of', count=7),
 Row(word='a', count=7),
 Row(word='in', count=5),
 Row(word='distributed', count=5)]

In [ ]:
spark.stop()